In [1]:
import os
import json
import numpy as np
import pandas as pd
import csv
import tqdm
import re
from itertools import combinations, product
from matplotlib import pyplot as plt
import seaborn as sns
from src.utils import *
%matplotlib inline

In [2]:
tqdm.tqdm.pandas()

C:\Users\Andrey\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
DATA_PATH = 'data'
STG_PATH = 'stg'
ZIP_FILE = 'data-science-bowl-2019.zip'
TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')
TEST_PATH = os.path.join(DATA_PATH, 'test.csv')
SPECS_PATH = os.path.join(DATA_PATH, 'specs.csv')
TRAIN_LABELS_PATH = os.path.join(DATA_PATH, 'train_labels.csv')
TEST_IDS_SCORE_PATH = os.path.join(STG_PATH, 'test_ids_score.csv')
TRAIN_LABELED_PATH = os.path.join(STG_PATH, 'train_labeled.csv')
TEST_LABELS_PATH = os.path.join(STG_PATH, 'test_labels.csv')
TRAIN_LABELS_EXTEND_PATH = os.path.join(STG_PATH, 'train_labels_extend.csv')
TEST_LABELED_PATH = os.path.join(STG_PATH, 'test_labeled.csv')
TRAIN_LABELED_EXTEND_PATH = os.path.join(STG_PATH, 'train_labeled_extend.csv')

# unzip(os.path.join(DATA_PATH, zip_file), data_path)

In [4]:
os.listdir(DATA_PATH)

['data-science-bowl-2019.zip',
 'sample_submission.csv',
 'specs.csv',
 'test.csv',
 'train.csv',
 'train_labels.csv']

### 1. Select installation_id and game_session from test_set for prediction

In [39]:
test_raw = pd.read_csv(TEST_PATH)
test_raw.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [40]:
test_ids_score = test_raw \
    .sort_values(['installation_id', 'timestamp']) \
    .groupby(['installation_id']) \
    .last() \
    .reset_index() \
    .loc[:, ['game_session', 'installation_id', 'title']]
test_ids_score.head()

,game_session,installation_id,title
0,348d7f09f96af313,00abaee7,Cauldron Filler (Assessment)
1,1fef5d54cb4b775a,01242218,Cart Balancer (Assessment)
2,4b165a330a0bdd6c,017c5718,Mushroom Sorter (Assessment)
3,be0b655ad1fee30c,01a44906,Mushroom Sorter (Assessment)
4,46e8bbed71df7520,01bc6cb6,Cart Balancer (Assessment)


In [7]:
test_ids_score.to_csv(TEST_IDS_SCORE_PATH, index=False)

### 2. Clean train set from installation_id without assessment attempts

In [41]:
train_raw = pd.read_csv(TRAIN_PATH)
train_raw.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [42]:
train_labels = pd.read_csv(TRAIN_LABELS_PATH)
train_labels.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [43]:
u_train_labels = train_labels['installation_id'].unique()
train_labeled = train_raw[train_raw['installation_id'].isin(u_train_labels)]

In [11]:
train_labeled.to_csv(TRAIN_LABELED_PATH, index=False)

### 3. Select installation_id and game_session with assessment attempts. They can be used in training.

In [5]:
def get_attempt_result(event_data_str):
    event_data_json = json.loads(event_data_str)
    if 'correct' in event_data_json:
        attempt_result = int(event_data_json['correct'])
    else:
        attempt_result = 0
    return attempt_result

In [8]:
test_raw['correct'] = test_raw['event_data'].progress_apply(get_attempt_result)

In [6]:
def set_assessment_result(df):
    p_type = (df['type'] == 'Assessment')
    code_4100 = (df['event_code'] == 4100)
    code_4110 = (df['event_code'] == 4110)
    bird_measurer = (df['title'] == 'Bird Measurer (Assessment)')
    attempts = p_type & (code_4100 & ~bird_measurer | code_4110 & bird_measurer)
    df['correct'] = attempts * df['correct']
    df['incorrect'] = attempts * (1 - df['correct'])

In [7]:
def accuracy(num_correct, num_incorrect):
    if num_correct == 1 and num_incorrect == 0:
        return 1
    elif num_correct == 0 and num_incorrect == 0:
        return -1
    else:
        return num_correct / (num_correct + num_incorrect)

In [8]:
def accuracy_group(accuracy):
    if accuracy == 0:
        return 0
    elif accuracy > 0 and accuracy < 0.5:
        return 1
    elif accuracy == 0.5:
        return 2
    elif accuracy == 1:
        return 3
    else:
        return -1

In [9]:
def get_labels(df):
    if df.shape[0] == 0:
        df_labels = pd.DataFrame(
            columns=['game_session', 'installation_id', 'title', 'num_correct', 'num_incorrect', 'accuracy', 'accuracy_group']
        )
        return df_labels
    df_labels = df \
        .groupby(['game_session', 'installation_id', 'title'])[['correct', 'incorrect']] \
        .sum() \
        .reset_index()

    df_labels.rename({'correct': 'num_correct', 'incorrect': 'num_incorrect'}, axis=1, inplace=True)

    df_labels['accuracy'] = df_labels[['num_correct', 'num_incorrect']].apply(
        lambda x: accuracy(x['num_correct'], x['num_incorrect']),
        axis=1
    )

    df_labels['accuracy_group'] = df_labels['accuracy'].apply(accuracy_group)
    return df_labels

In [50]:
set_assessment_result(test_raw)
test_labels = get_labels(test_raw)
test_labels = test_labels[test_labels['accuracy_group'] != -1]
test_labels.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
5,001a139acd7fce92,285b65c8,Bird Measurer (Assessment),1,15,0.0625,1
18,003bafadf2259738,56a739ec,Bird Measurer (Assessment),1,0,1.0000,3
40,006b058c3bba11a2,d95dc7cb,Mushroom Sorter (Assessment),1,0,1.0000,3
44,007195dd54b7c251,6cd8f2d8,Cart Balancer (Assessment),1,0,1.0000,3
48,007b28de31382377,ce08e98b,Mushroom Sorter (Assessment),0,1,0.0000,0


In [51]:
train_labels_extend = pd.concat([train_labels, test_labels], ignore_index=True, sort=False)
train_labels_extend.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [20]:
test_labels.to_csv(TEST_LABELS_PATH, index=False)
train_labels_extend.to_csv(TRAIN_LABELS_EXTEND_PATH, index=False)

### 4. Making addition to train set of labeled sessions from test set

In [21]:
labeled_sessions_flg = test_raw \
    .loc[:, ['game_session', 'installation_id']] \
    .apply(tuple, axis=1) \
    .isin(
        test_labels \
            .loc[:, ['game_session', 'installation_id']] \
            .apply(tuple, axis=1)
    )

In [22]:
labeled_sessions = test_raw[labeled_sessions_flg]
labeled_sessions.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,correct,incorrect
282,7ad3efc6,8b38fc0d2fd315dc,2019-09-11T18:56:11.918Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",00abaee7,1,2000,0,Cart Balancer (Assessment),Assessment,CRYSTALCAVES,0,0
283,65a38bf7,8b38fc0d2fd315dc,2019-09-11T18:56:12.019Z,"{""crystals"":[{""id"":""gem07"",""weight"":6},{""id"":""...",00abaee7,2,2020,58,Cart Balancer (Assessment),Assessment,CRYSTALCAVES,0,0
284,795e4a37,8b38fc0d2fd315dc,2019-09-11T18:56:12.030Z,"{""description"":""We've got to balance this scal...",00abaee7,3,3010,58,Cart Balancer (Assessment),Assessment,CRYSTALCAVES,0,0
285,5e109ec3,8b38fc0d2fd315dc,2019-09-11T18:56:15.790Z,"{""crystal_id"":""gem07"",""weight"":6,""coordinates""...",00abaee7,4,4030,3852,Cart Balancer (Assessment),Assessment,CRYSTALCAVES,0,0
286,5c2f29ca,8b38fc0d2fd315dc,2019-09-11T18:56:17.869Z,"{""crystal_id"":""gem07"",""weight"":6,""coordinates""...",00abaee7,5,4020,5968,Cart Balancer (Assessment),Assessment,CRYSTALCAVES,0,0


In [23]:
test_max_dates = labeled_sessions \
    .groupby('installation_id')['timestamp'] \
    .max() \
    .reset_index()
test_max_dates.head()

,installation_id,timestamp
0,00abaee7,2019-09-11T18:56:41.937Z
1,01242218,2019-09-22T22:29:09.638Z
2,02256298,2019-08-03T10:56:08.927Z
3,027e7ce5,2019-09-23T19:20:21.621Z
4,02a29f99,2019-08-17T18:52:02.446Z


In [24]:
test_labeled = pd.merge(test_raw, test_max_dates, how='inner', on=['installation_id'], suffixes=('', '_max'))
test_labeled.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,correct,incorrect,timestamp_max
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,0,0,2019-09-11T18:56:41.937Z
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,0,0,2019-09-11T18:56:41.937Z
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK,0,0,2019-09-11T18:56:41.937Z
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES,0,0,2019-09-11T18:56:41.937Z
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES,0,0,2019-09-11T18:56:41.937Z


In [25]:
test_labeled = test_labeled \
    .loc[test_labeled['timestamp'] <= test_labeled['timestamp_max']] \
    .drop(['correct', 'incorrect', 'timestamp_max'], axis=1)
test_labeled.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [28]:
test_labeled.to_csv(TEST_LABELED_PATH, index=False)

In [29]:
train_labeled_extend = pd.concat([train_labeled, test_labeled], sort=False, ignore_index=True)
train_labeled_extend.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,34ba1a28d02ba8ba,2019-08-06T04:57:18.904Z,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,4b57c9a59474a1b9,2019-08-06T04:57:45.301Z,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,2b9d5af79bcdb79f,2019-08-06T04:58:14.538Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,2b9d5af79bcdb79f,2019-08-06T04:58:14.615Z,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,2,3010,29,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1325467d,2b9d5af79bcdb79f,2019-08-06T04:58:16.680Z,"{""coordinates"":{""x"":273,""y"":650,""stage_width"":...",0006a69f,3,4070,2137,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [30]:
train_labeled_extend.to_csv(TRAIN_LABELED_EXTEND_PATH, index=False)

### 5. Feature engineering

In [10]:
train_labels = pd.read_csv(TRAIN_LABELS_PATH)
train_labeled = pd.read_csv(TRAIN_LABELED_PATH)
train_labeled['timestamp'] = pd.to_datetime(train_labeled['timestamp'])

In [11]:
train_labeled.sort_values(['installation_id', 'timestamp'], inplace=True)

In [12]:
cols = ('title', 'type', 'world')
group_cols = ('type', 'world')
cols_unique_dict = dict()
for c in cols:
    unique_values = list(train_labeled[c].unique())
    if c == 'world':
        unique_values.remove('NONE')
    cols_unique_dict[c] = unique_values
cols_unique_dict[group_cols] = list(product(cols_unique_dict[group_cols[0]], cols_unique_dict[group_cols[1]]))

In [120]:
def get_features(df, labels_row, cols_unique_dict):

    EPS = 1e-10
    features_dict = dict()

    # assessment identification in training set
    features_dict['installation_id'] = labels_row['installation_id']
    features_dict['game_session'] = labels_row['game_session']

    # flags for selection of assessment row in raw data
    installation_id_flgs = (df['installation_id'] == features_dict['installation_id'])
    game_session_flgs = (df['game_session'] == features_dict['game_session'])
    start_session_flgs = (df['event_code'] == 2000)

    # assessment info
    assessment_info = df[installation_id_flgs & game_session_flgs & start_session_flgs]
    assessment_index = assessment_info.index[0]

#    features of assessment
    features_dict['title'] = assessment_info.loc[assessment_index, 'title']
    features_dict['world'] = assessment_info.loc[assessment_index, 'world']
    features_dict['year'] = assessment_info.loc[assessment_index, 'timestamp'].year
    features_dict['quarter'] = assessment_info.loc[assessment_index, 'timestamp'].quarter
    features_dict['month'] = assessment_info.loc[assessment_index, 'timestamp'].month
    features_dict['day'] = assessment_info.loc[assessment_index, 'timestamp'].day
    features_dict['hour'] = assessment_info.loc[assessment_index, 'timestamp'].hour
    features_dict['minute'] = assessment_info.loc[assessment_index, 'timestamp'].minute
    features_dict['second'] = assessment_info.loc[assessment_index, 'timestamp'].second
    features_dict['dayofweek'] = assessment_info.loc[assessment_index, 'timestamp'].dayofweek
    features_dict['weekend'] = 1 if features_dict['dayofweek'] in (5, 6) else 0

    # history before assessment
    history_flgs = df['timestamp'] < assessment_info.loc[assessment_index, 'timestamp']
    before_assessment = df[installation_id_flgs & history_flgs].copy()
    
    before_assessment['correct'] = before_assessment['event_data'].apply(get_attempt_result)
    set_assessment_result(before_assessment)
    before_assessment_labels = get_labels(before_assessment)
    before_assessment_labels = before_assessment_labels[before_assessment_labels['accuracy_group'] != -1]

    game_session_cnt = before_assessment \
        .loc[:, 'game_session'] \
        .nunique()
    features_dict['game_sessions_cnt'] = game_session_cnt

    no_0_game_time_flgs = (before_assessment['game_time'] != 0)
    game_session_cnt_no_0_game_time = before_assessment \
        .loc[no_0_game_time_flgs, 'game_session'] \
        .nunique()
    features_dict['game_sessions_cnt_no_0_game_time'] = game_session_cnt_no_0_game_time
    
    game_sessions_time = before_assessment \
        .groupby('game_session')['timestamp'] \
        .agg(['max', 'min'])
    game_sessions_time = (game_sessions_time['max'] - game_sessions_time['min']).dt.seconds
#         game_sessions_time_stats = game_sessions_time.describe()
#         for s in game_sessions_time_stats.index:
#             if s != 'count':
#                 features_dict[f'game_sessions_time_{s}'] = game_sessions_time_stats[s]
    features_dict['game_sessions_time_mean'] = game_sessions_time.mean()
    features_dict['game_sessions_time_median'] = game_sessions_time.median()

    game_sessions_no_0_game_time = before_assessment \
        .loc[no_0_game_time_flgs] \
        .groupby('game_session')['timestamp'] \
        .agg(['max', 'min'])
    game_sessions_no_0_game_time = (game_sessions_no_0_game_time['max'] - game_sessions_no_0_game_time['min']).dt.seconds
    features_dict['game_sessions_no_0_game_time_mean'] = game_sessions_no_0_game_time.mean()
    features_dict['game_sessions_no_0_game_time_median'] = game_sessions_no_0_game_time.median()
    
    features_dict['num_correct_sum'] = before_assessment_labels['num_correct'].sum()
    features_dict['num_correct_mean'] = before_assessment_labels['num_correct'].mean()
    features_dict['num_correct_median'] = before_assessment_labels['num_correct'].median()
    features_dict['num_incorrect_sum'] = before_assessment_labels['num_incorrect'].sum()
    features_dict['num_incorrect_mean'] = before_assessment_labels['num_incorrect'].mean()
    features_dict['num_incorrect_median'] = before_assessment_labels['num_incorrect'].median()
    features_dict['accuracy_mean'] = before_assessment_labels['accuracy'].mean()
    features_dict['accuracy_median'] = before_assessment_labels['accuracy'].median()
    features_dict['accuracy_group_mean'] = before_assessment_labels['accuracy_group'].mean()
    features_dict['accuracy_group_median'] = before_assessment_labels['accuracy_group'].median()
    features_dict['accuracy_before_assessment'] = accuracy(
        features_dict['num_correct_sum'],
        features_dict['num_incorrect_sum']
    )
    features_dict['accuracy_group_before_assessment'] = accuracy_group(features_dict['accuracy_before_assessment'])
    
    features_dict['num_correct_all_sessions_div'] = np.log(
        (features_dict['num_correct_sum'] + EPS) / (features_dict['game_sessions_cnt'] + EPS)
    )
    features_dict['num_incorrect_all_sessions_div'] = np.log(
        (features_dict['num_incorrect_sum'] + EPS) / (features_dict['game_sessions_cnt'] + EPS)
    )
    if features_dict['game_sessions_cnt'] != 0:
        features_dict['accuracy_before_assessment_all_sessions_part'] = (
            features_dict['accuracy_before_assessment'] / \
            features_dict['game_sessions_cnt']
        )
    else:
        features_dict['accuracy_before_assessment_all_sessions_part'] = 0
    features_dict['accuracy_group_before_assessment_all_sessions_part'] = accuracy_group(
        features_dict['accuracy_before_assessment_all_sessions_part']
    )
    
    
    for cud in cols_unique_dict:
        for s in cols_unique_dict[cud]:
            
            if type(cud) is str:    
                game_session_slice_cnt = before_assessment[before_assessment[cud] == s]
                feature_name = f'{cud}_{s}'
            
            elif type(cud) is tuple:   
                game_session_slice_cnt = before_assessment.loc[np.all(before_assessment.loc[:, cud].values == s, axis=1)]
                feature_name = '_'.join(cud) + '_' + '_'.join(s)
                
            features_dict[f'game_sessions_{feature_name}_cnt'] = game_session_slice_cnt \
                .loc[:, 'game_session'] \
                .nunique()
            if features_dict['game_sessions_cnt'] == 0:
                features_dict[f'game_sessions_{feature_name}_cnt_part'] = 0
            else:
                features_dict[f'game_sessions_{feature_name}_cnt_part'] = \
                    features_dict[f'game_sessions_{feature_name}_cnt'] / features_dict['game_sessions_cnt']
                
            features_dict[f'game_sessions_no_0_game_time_{feature_name}_cnt'] = game_session_slice_cnt \
                .loc[no_0_game_time_flgs, 'game_session'] \
                .nunique()
            if features_dict['game_sessions_cnt_no_0_game_time'] == 0:
                features_dict[f'game_sessions_no_0_game_time_{feature_name}_cnt_part'] = 0
            else:
                features_dict[f'game_sessions_no_0_game_time_{feature_name}_cnt_part'] = \
                    features_dict[f'game_sessions_no_0_game_time_{feature_name}_cnt'] / \
                    features_dict['game_sessions_cnt_no_0_game_time']
            
            game_session_slice_time = game_session_slice_cnt \
                .groupby('game_session')['timestamp'] \
                .agg(['max', 'min'])
            game_session_slice_time = (game_session_slice_time['max'] - game_session_slice_time['min']).dt.seconds
#             game_session_slice_time_stats = game_session_slice_time.describe()
#             for st in game_session_slice_time_stats.index:
#                 if st != 'count':
#                     features_dict[f'game_sessions_time_{cud}_{s}_{st}'] = game_sessions_time_slice_stats[st]
#                     features_dict[f'game_sessions_time_part_{cud}_{s}_{st}'] = np.log(
#                         (features_dict[f'game_sessions_time_{cud}_{s}_{st}'] + eps) / \
#                         (features_dict[f'game_sessions_time_{st}'] + eps)
#                     )
            features_dict[f'game_sessions_time_{feature_name}_mean'] = game_session_slice_time.mean()
            features_dict[f'game_sessions_time_{feature_name}_mean_div'] = np.log(
                (features_dict[f'game_sessions_time_{feature_name}_mean'] + EPS) / \
                (features_dict[f'game_sessions_time_mean'] + EPS)
            )
            features_dict[f'game_sessions_time_{feature_name}_median'] = game_session_slice_time.median()
            features_dict[f'game_sessions_time_{feature_name}_median_div'] = np.log(
                (features_dict[f'game_sessions_time_{feature_name}_median'] + EPS) / \
                (features_dict[f'game_sessions_time_median'] + EPS)
            )

            game_session_slice_no_0_game_time = game_session_slice_cnt \
                .loc[no_0_game_time_flgs] \
                .groupby('game_session')['timestamp'] \
                .agg(['max', 'min'])
            game_session_slice_no_0_game_time = (
                game_session_slice_no_0_game_time['max'] - game_session_slice_no_0_game_time['min']
            ).dt.seconds
            features_dict[f'game_sessions_no_0_game_time_{feature_name}_mean'] = game_session_slice_no_0_game_time.mean()
            features_dict[f'game_sessions_no_0_game_time_{feature_name}_mean_div'] = np.log(
                (features_dict[f'game_sessions_no_0_game_time_{feature_name}_mean'] + EPS) / \
                (features_dict[f'game_sessions_no_0_game_time_mean'] + EPS)
            )
            features_dict[f'game_sessions_no_0_game_time_{feature_name}_median'] = game_session_slice_no_0_game_time \
                .median()
            features_dict[f'game_sessions_no_0_game_time_{feature_name}_median_div'] = np.log(
                (features_dict[f'game_sessions_no_0_game_time_{feature_name}_median'] + EPS) / \
                (features_dict[f'game_sessions_no_0_game_time_median'] + EPS)
            )

            if 'Assessment' in feature_name:
                assessment_session_slice_cnt = game_session_slice_cnt.copy()
                assessment_session_slice_cnt['correct'] = assessment_session_slice_cnt['event_data'].apply(get_attempt_result)
                set_assessment_result(assessment_session_slice_cnt)
                assessment_session_slice_cnt_labels = get_labels(assessment_session_slice_cnt)
                assessment_session_slice_cnt_labels = assessment_session_slice_cnt_labels[
                    assessment_session_slice_cnt_labels['accuracy_group'] != -1
                ]

                features_dict[f'num_correct_{feature_name}_sum'] = assessment_session_slice_cnt_labels['num_correct'].sum()
                if features_dict[f'num_correct_{feature_name}_sum'] == 0:
                     features_dict[f'num_correct_{feature_name}_sum_part'] = 0
                else:
                    features_dict[f'num_correct_{feature_name}_sum_part'] = \
                        features_dict[f'num_correct_{feature_name}_sum'] / \
                        features_dict['num_correct_sum']
                features_dict[f'num_correct_{feature_name}_mean'] = assessment_session_slice_cnt_labels['num_correct'].mean()
                features_dict[f'num_correct_{feature_name}_median'] = assessment_session_slice_cnt_labels['num_correct'].median()
                features_dict[f'num_incorrect_{feature_name}_sum'] = assessment_session_slice_cnt_labels['num_incorrect'].sum()
                if features_dict[f'num_incorrect_{feature_name}_sum'] == 0:
                    features_dict[f'num_incorrect_{feature_name}_sum_part'] = 0
                else:
                    features_dict[f'num_incorrect_{feature_name}_sum_part'] = \
                        features_dict[f'num_incorrect_{feature_name}_sum'] / \
                        features_dict['num_incorrect_sum']
                features_dict[f'num_incorrect_{feature_name}_mean'] = assessment_session_slice_cnt_labels['num_incorrect'].mean()
                features_dict[f'num_incorrect_{feature_name}_median'] = assessment_session_slice_cnt_labels['num_incorrect'].median()
                features_dict[f'accuracy_{feature_name}'] = accuracy(
                    features_dict[f'num_correct_{feature_name}_sum'],
                    features_dict[f'num_incorrect_{feature_name}_sum']
                )
                features_dict[f'accuracy_{feature_name}_mean'] = assessment_session_slice_cnt_labels['accuracy'].mean()
                features_dict[f'accuracy_{feature_name}_median'] = assessment_session_slice_cnt_labels['accuracy'].median()
                features_dict[f'accuracy_group_{feature_name}_mean'] = assessment_session_slice_cnt_labels['accuracy_group'].mean()
                features_dict[f'accuracy_group_{feature_name}_median'] = assessment_session_slice_cnt_labels['accuracy_group'].median()
    
    features_dict['num_correct_assessment_sessions_div'] = np.log(
        (features_dict['num_correct_sum'] + EPS) / (features_dict['game_sessions_type_Assessment_cnt'] + EPS)
    )
    features_dict['num_incorrect_assessment_sessions_div'] = np.log(
        (features_dict['num_incorrect_sum'] + EPS) / (features_dict['game_sessions_type_Assessment_cnt'] + EPS)
    )
    if features_dict['game_sessions_type_Assessment_cnt'] != 0:
        features_dict['accuracy_before_assessment_assessment_sessions_part'] = (
            features_dict['accuracy_before_assessment'] / features_dict['game_sessions_type_Assessment_cnt']
        )
    else:
        features_dict['accuracy_before_assessment_assessment_sessions_part'] = 0
    features_dict['accuracy_group_before_assessment_assessment_sessions_part'] = accuracy_group(
        features_dict['accuracy_before_assessment_assessment_sessions_part']
    )

    return features_dict

tmp = get_features(train_labeled, train_labels.iloc[0], cols_unique_dict)

In [121]:
len(tmp)

912

In [122]:
df = train_labels.iloc[2:3, :].apply(lambda x: get_features(train_labeled, x, cols_unique_dict), axis=1)
df = pd.DataFrame(list(df))

In [124]:
[re.findall('num_incorrect.*sum.*', x)[0] for x in df.columns if len(re.findall('num_incorrect.*sum.*', x)) != 0]

['num_incorrect_sum',
 'num_incorrect_title_Mushroom Sorter (Assessment)_sum',
 'num_incorrect_title_Mushroom Sorter (Assessment)_sum_part',
 'num_incorrect_title_Bird Measurer (Assessment)_sum',
 'num_incorrect_title_Bird Measurer (Assessment)_sum_part',
 'num_incorrect_title_Cauldron Filler (Assessment)_sum',
 'num_incorrect_title_Cauldron Filler (Assessment)_sum_part',
 'num_incorrect_title_Cart Balancer (Assessment)_sum',
 'num_incorrect_title_Cart Balancer (Assessment)_sum_part',
 'num_incorrect_title_Chest Sorter (Assessment)_sum',
 'num_incorrect_title_Chest Sorter (Assessment)_sum_part',
 'num_incorrect_type_Assessment_sum',
 'num_incorrect_type_Assessment_sum_part',
 'num_incorrect_type_world_Assessment_MAGMAPEAK_sum',
 'num_incorrect_type_world_Assessment_MAGMAPEAK_sum_part',
 'num_incorrect_type_world_Assessment_TREETOPCITY_sum',
 'num_incorrect_type_world_Assessment_TREETOPCITY_sum_part',
 'num_incorrect_type_world_Assessment_CRYSTALCAVES_sum',
 'num_incorrect_type_world_As

In [125]:
df[[re.findall('num_correct.*sum.*', x)[0] for x in df.columns if len(re.findall('num_correct.*sum.*', x)) != 0]]

,num_correct_sum,num_correct_title_Mushroom Sorter (Assessment)_sum,num_correct_title_Mushroom Sorter (Assessment)_sum_part,num_correct_title_Bird Measurer (Assessment)_sum,num_correct_title_Bird Measurer (Assessment)_sum_part,num_correct_title_Cauldron Filler (Assessment)_sum,num_correct_title_Cauldron Filler (Assessment)_sum_part,num_correct_title_Cart Balancer (Assessment)_sum,num_correct_title_Cart Balancer (Assessment)_sum_part,num_correct_title_Chest Sorter (Assessment)_sum,num_correct_title_Chest Sorter (Assessment)_sum_part,num_correct_type_Assessment_sum,num_correct_type_Assessment_sum_part,num_correct_type_world_Assessment_MAGMAPEAK_sum,num_correct_type_world_Assessment_MAGMAPEAK_sum_part,num_correct_type_world_Assessment_TREETOPCITY_sum,num_correct_type_world_Assessment_TREETOPCITY_sum_part,num_correct_type_world_Assessment_CRYSTALCAVES_sum,num_correct_type_world_Assessment_CRYSTALCAVES_sum_part
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
